# Volatility Model Calibration

This notebook demonstrates **Time-Adjusted Wing Model calibration** with a streamlined approach.

## Features
- **Single Model Focus**: Time-Adjusted Wing Model only
- **Configuration-Driven**: Respects config settings
- **Simplified Flow**: Reduced complexity and cleaner cells
- **Visual Results**: Clear performance visualization

## 1. Setup

In [1]:
# Essential imports
import sys, os, warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# Project setup
sys.path.append(project_root:=os.path.dirname(os.getcwd()))

# Use the new improved configuration system
from config import VolatilityConfig
config = VolatilityConfig()

# Extract configuration values with fallbacks for notebook-specific settings
date_str = config.get('data.date_str', '20240229')
option_expiry = '8MAR24'  # This might need to be added to config or derived from data
snapshot_time_str = '2024-02-29T20:22:00'  # This might need to be added to config

print(f"📋 Using expiry: {option_expiry}, date: {date_str}")
print(f"✅ Time-Adjusted Wing Model enabled: {config.time_adjusted_wing_model_enabled}")

📋 Using expiry: 8MAR24, date: 20240229
✅ Time-Adjusted Wing Model enabled: True


## 2. Data Loading

In [2]:
# Data Loading & Pipeline Setup 🚀
from utils.market_data.orderbook_deribit_md_manager import OrderbookDeribitMDManager
import polars as pl

data_file = os.path.join(project_root, config.get_data_file_path())
print(f"📂 Loading: {data_file}")

df_raw = pl.scan_csv(data_file).with_columns(pl.col('index_price').cast(pl.Float64))
symbol_manager = OrderbookDeribitMDManager(df_raw, date_str, config)

📂 Loading: /home/user/Python/Baseoffset-and-Volatility-Fitting-Manager/data_orderbook/20240229.output.csv.gz


In [ ]:
# Load market data
from utils.volatility_fitter.processed_data_loader import load_baseoffset_results

# Use config values
snapshot_time = datetime.strptime(snapshot_time_str, "%Y-%m-%dT%H:%M:%S")
my_expiry = option_expiry

# Data Conflation & Symbol Analysis 🔄
print(f"🔄 Conflating data (every {(conflation_every:='10s')}, period {(conflation_period:='5m')})...")
df_conflated_md = symbol_manager.get_conflated_md(freq=conflation_every, period=conflation_period).sort(['expiry', 'timestamp'])

# load previous baseoffset results
df_baseoffset = load_baseoffset_results(date_str)

# join baseoffset results to conflated md
df_conflated_md = df_conflated_md.with_columns(
    option_type=pl.when(pl.col('is_call')).then(pl.lit('C')).otherwise(pl.lit('P'))
).select(['symbol', 'timestamp', 'expiry', 'strike', 'bid_size', 'bid_price', 
          'ask_price', 'ask_size', 'S', 'expiry_ts', 'option_type', 'tau']  
).join(df_baseoffset.select(['expiry', 'timestamp', 'F', 'r']),
       on=['timestamp', 'expiry']
).sort(['expiry_ts', 'strike', 'option_type'])

display(df_conflated_md.head())

🔄 Conflating data (every 10s, period 5m)...
🔄 Converting order book depth data to BBO format (using level 0)...
✅ Converted 8194008 orderbook rows to 8194008 BBO rows
🔄 Normalizing volume from USD to BTC for futures and perpetuals...
✅ Normalized volumes for 8/8 futures/perpetual symbols


symbol,timestamp,expiry,strike,bid_size,bid_price,ask_price,ask_size,S,expiry_ts,option_type,tau,F,r
str,datetime[μs],str,i64,f64,f64,f64,f64,f64,datetime[μs],str,f64,f64,f64
"""BTC-29FEB24-45000-P""",2024-02-29 00:17:00,"""29FEB24""",45000,null,null,0.0004,13.0,61969.64,2024-02-29 08:00:00,"""P""",0.000881,61988.23,0.3982
"""BTC-29FEB24-45000-P""",2024-02-29 00:15:30,"""29FEB24""",45000,null,null,0.0004,13.0,61892.42,2024-02-29 08:00:00,"""P""",0.000884,61910.99,0.3973
"""BTC-29FEB24-45000-P""",2024-02-29 01:58:00,"""29FEB24""",45000,null,null,0.0001,0.1,61353.35,2024-02-29 08:00:00,"""P""",0.000689,61365.62,0.384
"""BTC-29FEB24-45000-P""",2024-02-29 01:20:30,"""29FEB24""",45000,null,null,0.0001,0.1,61370.01,2024-02-29 08:00:00,"""P""",0.00076,61382.29,0.393
"""BTC-29FEB24-45000-P""",2024-02-29 00:15:00,"""29FEB24""",45000,null,null,0.0085,15.0,61774.85,2024-02-29 08:00:00,"""P""",0.000885,61793.39,0.3996


In [4]:
print(f"Conflated MD: {len(df_conflated_md)}, Min Timestamp: {df_conflated_md['timestamp'].min()}, Max Timestamp: {df_conflated_md['timestamp'].max()}")

Conflated MD: 2851826, Min Timestamp: 2024-02-29 00:12:00, Max Timestamp: 2024-02-29 23:59:00


In [ ]:
# Get the Snapshot MD for Option Chain Processing and Volatility Fitting📊
from utils.reporting.html_table_generator import generate_price_comparison_table
from utils.volatility_fitter.volatility_calculator import process_option_chain_with_volatilities, process_volatility_with_greeks
from IPython.display import HTML, display

rate = df_baseoffset.filter(pl.col('expiry')==my_expiry, pl.col('timestamp')==snapshot_time)['r'][0]
print(f"interest rate for expiry {my_expiry} @ {snapshot_time} = {rate:.3f}")

df_conflated_md = df_conflated_md.with_columns(is_call = pl.col('option_type') == 'C', is_put = pl.col('option_type') == 'P',
        bid_price_fut = pl.col('F'),ask_price_fut = pl.col('F'))
tightened_option_chain, _ = symbol_manager.create_option_synthetic(df_conflated_md, my_expiry, snapshot_time, 
                                                                   config.tightening_volume_threshold, rate)
display(HTML(generate_price_comparison_table(tightened_option_chain, table_width="70%", font_size="10px", volume_threshold=config.tightening_volume_threshold)))
df_option_with_vola = process_option_chain_with_volatilities(tightened_option_chain, interest_rate=rate)
df_option_with_vola_and_greeks = process_volatility_with_greeks(df_option_with_vola)


interest rate for expiry 8MAR24 @ 2024-02-29 20:22:00 = 0.314
📝 Generating HTML price comparison table for the tightened bid/ask spread on call and put options... (Volume Threshold: 5.0)


In [14]:
# # Process option data
import numpy as np

display(df_option_with_vola_and_greeks.head())
# Extract key parameters
forward_price = df_option_with_vola['F'][0]
time_to_expiry = df_option_with_vola['tau'][0]
strikes_list = df_option_with_vola_and_greeks['strike'].to_list()
market_vols = (df_option_with_vola_and_greeks['midVola']/100).to_list()
market_vegas = df_option_with_vola_and_greeks['vega'].to_list()

# Use config for weighting scheme
if config.weighting_scheme == 'vega':
    weight_arr = np.array(market_vegas)/max(market_vegas)
    print(f"⚖️ Using {config.weighting_scheme} weighting scheme from config")
else:
    weight_arr = np.ones(len(market_vegas))  # Equal weighting fallback
    print(f"⚖️ Using equal weighting (config scheme '{config.weighting_scheme}' not implemented)")

print(f"💰 Forward: ${forward_price:,.2f}, ⏰ Time to expiry: {time_to_expiry:.4f} years")
print(f"📈 {len(strikes_list)} strikes from ${min(strikes_list):,.0f} to ${max(strikes_list):,.0f}")

timestamp,bq0_C,bp0_C,bp0_C_usd,ap0_C_usd,ap0_C,aq0_C,strike,bq0_P,bp0_P,bp0_P_usd,ap0_P_usd,ap0_P,aq0_P,S,F,expiry,tau,r,bidVola_C,askVola_C,bidVola_P,askVola_P,bidVola,askVola,midVola,volSpread,vega,call_delta
datetime[μs],f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-02-29 20:22:00,6.4,0.316,19604.26,20416.51,0.329093,6.4,42000,7.5,0.0003,18.61,31.02,0.0005,7.9,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,160.563226,112.235346,119.507368,112.24,119.51,115.88,7.27,1.7,0.99
2024-02-29 20:22:00,6.4,0.284,17619.02,18429.35,0.297062,6.4,44000,0.2,0.0005,31.02,43.43,0.0007,10.2,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,144.461441,106.830169,111.8134,106.83,111.81,109.32,4.98,2.49,0.98
2024-02-29 20:22:00,6.3,0.268,16626.4,17435.78,0.281046,6.3,45000,3.4,0.0005,31.02,49.63,0.0008,17.4,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,136.625904,100.669673,107.453753,100.67,107.45,104.06,6.78,2.74,0.98
2024-02-29 20:22:00,6.3,0.252,15633.78,16442.2,0.265031,6.3,46000,3.1,0.0006,37.22,55.83,0.0009,16.3,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,128.917551,96.998608,102.856956,97.0,102.86,99.93,5.86,3.17,0.98
2024-02-29 20:22:00,6.3,0.2365,14672.18,15448.62,0.249015,6.3,47000,4.0,0.0008,49.63,68.24,0.0011,15.3,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,121.327405,94.790857,99.556841,94.79,99.56,97.18,4.77,3.9,0.98


⚖️ Using vega weighting scheme from config
💰 Forward: $62,343.55, ⏰ Time to expiry: 0.0205 years
📈 41 strikes from $42,000 to $76,000


In [8]:
import plotly.graph_objects as go

fig = go.Figure()
# Plot with error bars for bid/ask implied volatility
fig.add_trace(go.Scatter(
    x=strikes_list,
    y=(df_option_with_vola_and_greeks['bidVola_C']+df_option_with_vola_and_greeks['askVola_C'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_C'] - df_option_with_vola_and_greeks['bidVola_C']).abs()/2,
        visible=True, color='blue'),
    mode='markers',
    name='Call IV (Bid/Ask Error Bar)',
    marker=dict(color='blue', symbol='circle'),
    opacity=0.8
))
fig.add_trace(go.Scatter(
    x=strikes_list,
    y=(df_option_with_vola_and_greeks['bidVola_P']+df_option_with_vola_and_greeks['askVola_P'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_P'] - df_option_with_vola_and_greeks['bidVola_P']).abs()/2,
        visible=True, color='orange'),
    mode='markers',
    name='Put IV (Bid/Ask Error Bar)',
    marker=dict(color='orange', symbol='circle'),
    opacity=0.8
))

fig.add_vline(x=df_option_with_vola_and_greeks['S'][0], line=dict(color='green', dash='dash'), name='Spot Price (S)')

fig.update_layout(
    title=f"{snapshot_time.strftime('%Y-%m-%d %H:%M')}: IV on DERIBIT BTC option for Expiry {my_expiry}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility (%)",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()

## 3. Model Calibration

In [9]:
# Updated imports - using specific calibrators directly
from utils.volatility_fitter.calibrators import GlobalVolatilityCalibrator
from utils.volatility_fitter.calibrators.local_calibrator import LocalVolatilityCalibrator  # Time-Adjusted Wing Model Calibration
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model import TimeAdjustedWingModel
from utils.volatility_fitter.time_adjusted_wing_model import create_time_adjusted_wing_model_from_result

print(f"🔧 Configuration loaded: Time-Adjusted Wing Model: {'ENABLED' if config.time_adjusted_wing_model_enabled else 'DISABLED'}| Calibration method: {config.calibration_method}")

🔧 Configuration loaded: Time-Adjusted Wing Model: ENABLED| Calibration method: SLSQP


### 3.1 Initial Guess

In [10]:
# Define loss value externally and initial parameters
import numpy as np
from utils.volatility_fitter.calibration_result import CalibrationResult
from utils.volatility_fitter.wing_model.wing_model_parameters import WingModelParameters


def find_objective_loss_value(my_calibrator: GlobalVolatilityCalibrator|LocalVolatilityCalibrator, 
                              model_params: TimeAdjustedWingModel, strikes_list: list[float], 
                              market_vols: list[float], market_vegas: list[float], weights: list[float]) -> float:
    return float(my_calibrator._objective_function(
        x=np.array(model_params.get_fitted_vol_parameter()), initial_params=model_params, 
        param_names=model_params.get_parameter_names(), strikes=strikes_list, market_volatilities=market_vols, 
        market_vegas=market_vegas, enforce_arbitrage_free=True, weights=weights))
    
# Initial parameters
initial_guess = [0.4991, -1.7540, 1.29318, 1.86, -0.093005, 1.4, 3.0, 4.87]
calibration_results: list[str, WingModelParameters, float, str] = []
my_local_calibrator = LocalVolatilityCalibrator(model_class=TimeAdjustedWingModel, enable_bounds=True)

my_initial_wing_model = create_time_adjusted_wing_model_from_result(result=initial_guess,
    forward_price=forward_price, ref_price=forward_price, time_to_expiry=time_to_expiry)

my_initial_result = CalibrationResult(
    success=False, optimization_method="Initial Setup", parameters=my_initial_wing_model, 
    error=find_objective_loss_value(my_local_calibrator, my_initial_wing_model, strikes_list, market_vols, market_vegas, weight_arr), 
    message="Initial parameters before calibration")

calibration_results.append(my_initial_result)
print(f"🎯 Initial result: {my_initial_result}")

🎯 Initial result: CalibrationResult(
  success=False,
  method='Initial Setup',
  parameters=WingModelParameters(vr=0.4991, sr=-1.7540, pc=1.2932, cc=1.8600, dc=-0.0930, uc=1.4000, dsm=3.0000, usm=4.8700),
  error=7.279213,
  message='Initial parameters before calibration'
  optimisation_result=None
  time_elapsed=0.000000 seconds
)


### 3.2 Local Optimizer (SLSQP and L-BFGS-B) - both need to provide an initial guess of the parameter

In [11]:
#### CALIBRATION OF TIME-ADJUSTED WING MODEL ####
methods = ["SLSQP", "L-BFGS-B"]
if config.time_adjusted_wing_model_enabled:
    print(f"🚀 Calibrating Time-Adjusted Wing Model with optimisation methods {methods}")
    print(f"🎛️ Config method preference: {config.calibration_method}")
    print(f"🔧 Max iterations: {config.max_calibration_iterations}")

    for method_name in methods:
        
        # Create appropriate calibrator based on method
        my_local_calibrator = LocalVolatilityCalibrator(
            model_class=TimeAdjustedWingModel, 
            method=method_name, 
            enable_bounds=config.enable_bounds
        )
        my_result = my_local_calibrator.calibrate(
            my_initial_wing_model, 
            strikes_list, 
            market_vols, 
            market_vegas, 
            my_initial_wing_model.get_parameter_bounds(),  # Use the bounds from parameter object
            config.enforce_arbitrage_free,  # Use config setting
            None, 
            weight_arr
        )
        print(f"📍 Using LocalVolatilityCalibrator for {method_name}")
                    
        calibration_results.append(my_result)
        print(f"Loss: {my_result.error:.8f}, Success: {my_result.success}, Message: {my_result.message}, {my_result.parameters}")
    print("✅ Calibration complete")

🚀 Calibrating Time-Adjusted Wing Model with optimisation methods ['SLSQP', 'L-BFGS-B']
🎛️ Config method preference: SLSQP
🔧 Max iterations: 1000
📍 Using LocalVolatilityCalibrator for SLSQP
Loss: 0.13389694, Success: True, Message: Optimization successful, WingModelParameters(vr=0.7004, sr=0.0408, pc=1.1205, cc=0.1640, dc=-0.0088, uc=1.4000, dsm=2.7727, usm=4.8700)
📍 Using LocalVolatilityCalibrator for L-BFGS-B
Loss: 0.13375394, Success: True, Message: Optimization successful, WingModelParameters(vr=0.6999, sr=0.0456, pc=1.2660, cc=0.1550, dc=0.0000, uc=1.4000, dsm=2.9576, usm=4.8700)
✅ Calibration complete


### 3.3 Glocal Optimizer (Differential Evolution) - dun require the initial guess as it will find the global minimum

In [12]:
config.get_parameter_bounds('time_adjusted_wing_model')

[(0.05, 5.0),
 (-2.0, 2.0),
 (-1.001, 5.0),
 (0.001, 5.0),
 (-5.0, 0.0),
 (0.0, 5.0),
 (0.01, 10.0),
 (0.01, 10.0)]

In [13]:
# Differential Evolution - Global Optimization
if config.time_adjusted_wing_model_enabled :
    print("🧬 Running Differential Evolution (Global Optimization)...")
    
    try:
        # Create DE calibrator with specific settings for performance
        my_global_calibrator = GlobalVolatilityCalibrator(model_class=TimeAdjustedWingModel, enable_bounds=True, workers=5, config_loader=config)
        
        # Use parameter bounds from configuration
        param_bounds_config = config.get_parameter_bounds('time_adjusted_wing_model')
        if param_bounds_config:
            param_bound = param_bounds_config
            print(f"📐 Using parameter bounds from config: {len(param_bound)} parameters")
        else:
            raise ValueError("Parameter bounds for Time-Adjusted Wing Model not found in configuration.")

        # Run Differential Evolution with optimized parameters for speed vs quality
        ta_de_result = my_global_calibrator.calibrate(
            initial_params=my_initial_wing_model,  # this is useless for DE but kept for interface consistency
            strikes=strikes_list,
            market_volatilities=market_vols,
            market_vegas=market_vegas,
            parameter_bounds=param_bound,
            enforce_arbitrage_free=config.enforce_arbitrage_free,
            popsize=20,  # Reduced population size for faster execution
            maxiter=500,  # Reduced iterations for faster execution
            seed=42,      # For reproducible results
            weights=weight_arr
        )
        calibration_results.append(ta_de_result)
        print(f"🎯 Config settings: arbitrage_free={config.enforce_arbitrage_free}, tolerance={config.calibration_tolerance}")
    except Exception as e:
        print(f"⚠️ DE error: {e}")
    print(f"Loss: {ta_de_result.error:.8f}, Success: {ta_de_result.success}, Message: {ta_de_result.message}, Parameters: {ta_de_result.parameters}")

🧬 Running Differential Evolution (Global Optimization)...
📐 Using parameter bounds from config: 8 parameters
🎯 Config settings: arbitrage_free=True, tolerance=1e-10
Loss: 0.04653899, Success: True, Message: Differential Evolution successful (nfev: 8660), Parameters: WingModelParameters(vr=0.6993, sr=0.0540, pc=0.2315, cc=0.1383, dc=-1.9540, uc=2.0750, dsm=8.9549, usm=4.6569)


In [15]:
# comparison of results

pl.DataFrame([result.to_dict() for result in calibration_results])

method,success,error,message,time_elapsed,num_evaluations,vr,sr,pc,cc,dc,uc,dsm,usm
str,bool,f64,str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""Initial Setup""",false,7.279213,"""Initial parameters before cali…",0.0,0,0.4991,-1.754,1.29318,1.86,-0.093005,1.4,3.0,4.87
"""SLSQP""",true,0.133897,"""Optimization successful""",0.319227,139,0.700393,0.040787,1.120523,0.163989,-0.008769,1.4,2.772691,4.87
"""L-BFGS-B""",true,0.133754,"""Optimization successful""",0.696851,315,0.699936,0.045593,1.265955,0.155008,0.0,1.4,2.957626,4.87
"""Differential Evolution""",true,0.046539,"""Differential Evolution success…",5.521707,8660,0.699263,0.054048,0.231464,0.138257,-1.953964,2.075024,8.954864,4.656866


In [16]:
# Visualize results - Compare all calibration methods
import numpy as np
import plotly.graph_objects as go

# Create plot
fig = go.Figure()

# Plot market data with error bars
fig.add_trace(go.Scatter(
    x=strikes_list, y=market_vols, mode='markers', name='Market IV',
    error_y=dict(type='data', symmetric=False,
        array=(df_option_with_vola_and_greeks['askVola']-df_option_with_vola_and_greeks['midVola']).to_numpy()/100,
        arrayminus=(df_option_with_vola_and_greeks['midVola']-df_option_with_vola_and_greeks['bidVola']).to_numpy()/100,
        visible=True, color='rgba(0,0,0,0.3)'
    ),
    marker=dict(size=8, color='black', symbol='circle'),
    hovertemplate='Strike: %{x}<br>Market Vol: %{y:.4f}<extra></extra>'
))

# Generate extended strike range for smooth curves
extended_strikes = np.linspace(min(strikes_list) * 0.9, max(strikes_list) * 1.1, 100)

# Plot each calibration result
subtitle_text = ""
for calibration_result in calibration_results:
    model = TimeAdjustedWingModel(calibration_result.parameters)
    model_vols = [model.calculate_volatility_from_strike(strike) for strike in extended_strikes]
    fig.add_trace(go.Scatter(
        x=extended_strikes, y=model_vols, mode='lines', name=(name:=calibration_result.optimization_method),
        line=dict(width=2, dash="dash" if name.startswith("Initial") else "solid"), opacity=0.8,
        hovertemplate=f'{name}<br>Strike: %{{x}}<br>Vol: %{{y:.4f}}<extra></extra>'))
    subtitle_text += f"{name}: Loss={calibration_result.error:.5f} &nbsp;&nbsp; "

# Forward price line
fig.add_vline(x=forward_price, line=dict(color='purple', dash='dot', width=2), annotation_text=f"Forward: {forward_price:.0f}")

# Layout
fig.update_layout(
    title=f"Optimisation Comparison @ {snapshot_time} - {my_expiry} Expiry" +
            f"<span style='font-size:12px'>  (Forward: {forward_price:.0f} | τ: {time_to_expiry:.4f})<br>" +
            f"<span style='font-size:10px'>{subtitle_text}</span>",
    xaxis_title='Strike Price', yaxis_title='Implied Volatility', width=1000, height=650, template='plotly_white', hovermode='closest',
    legend=dict(x=0.80, y=0.75, bgcolor='rgba(255,255,255,0.5)'),
    yaxis_range=(df_option_with_vola_and_greeks['bidVola'].min()/100*0.95, df_option_with_vola_and_greeks['askVola'].max()/100*1.15),
    xaxis=dict(range=[min(extended_strikes), max(extended_strikes)]),
    margin=dict(t=100)
)

# Add strike range lines if my_result is available
# if 'my_result' in locals() and my_result is not None and my_result.success:
#     # Extract the parameters from the best result (last one in the list before 'Initial Guess')
#     best_result_params = None
#     for name, params, _, _ in calibration_results:
#         if name != "Initial Guess":
#             best_result_params = params

#     if best_result_params:
#         model_for_ranges = TimeAdjustedWingModel(best_result_params)
#         for range_name, range_strike in model_for_ranges.get_strike_ranges().items():
#             fig.add_vline(x=range_strike, line=dict(color='gray', dash='dot', width=1))
    
fig.show()

## 5. Option Chain with Fitted Vol and Greeks

In [17]:
# Add Fitted Volatilities and Greeks to existing DataFrame
import polars as pl
from utils.pricer.black76_option_pricer import Black76OptionPricer

# Get best model
best_result = min([r for r in calibration_results if r.optimization_method != "Initial Setup"], key=lambda x: x.error)
best_model = TimeAdjustedWingModel(best_result.parameters)
print(f"🎯 Using: {best_result.optimization_method} (Error: {best_result.error:.6f})")

# Calculate fitted data for all strikes
fitted_data = []
for i, strike in enumerate(strikes_list):
    fitted_vol = best_model.calculate_volatility_from_strike(strike)
    
    if time_to_expiry > 0 and fitted_vol > 0:
        call_greeks = Black76OptionPricer(forward_price, strike, time_to_expiry, rate, fitted_vol).get_all_greeks('call')
        put_greeks = Black76OptionPricer(forward_price, strike, time_to_expiry, rate, fitted_vol).get_all_greeks('put')
        fitted_data.append([fitted_vol*100, call_greeks['delta'], call_greeks['gamma'], call_greeks['theta'], 
                           call_greeks['vega']/100, call_greeks['price'], put_greeks['price'], (fitted_vol-market_vols[i])*100] if market_vols[i] > 0 else 0)
    else:
        fitted_data.append([fitted_vol*100, 0, 0, 0, 0, 0, 0, (fitted_vol-market_vols[i])*100, 0])

# Add to DataFrame
fitted_arrays = list(zip(*fitted_data))
df_option_with_vola_and_greeks = df_option_with_vola_and_greeks.with_columns([
    pl.Series(name, data) for name, data in zip(
        ["fitVola", "delta", "gamma", "theta", "vega", "tv_C", "tv_P", "vol_diff"], fitted_arrays)])

display(df_option_with_vola_and_greeks.select(["strike","midVola","bp0_C_usd","tv_C","ap0_C_usd","bp0_P_usd","tv_P","ap0_P_usd","fitVola","vol_diff",
                                               "delta","gamma","vega"]))

🎯 Using: Differential Evolution (Error: 0.046539)


strike,midVola,bp0_C_usd,tv_C,ap0_C_usd,bp0_P_usd,tv_P,ap0_P_usd,fitVola,vol_diff,delta,gamma,vega
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
42000,115.88,19604.26,20245.883366,20416.51,18.61,32.985773,31.02,120.461614,4.581614,0.984884,0.000002,2.103391
44000,109.32,17619.02,18260.616837,18429.35,31.02,34.874641,43.43,108.502563,-0.817437,0.983485,0.000003,2.396683
45000,104.06,16626.4,17269.713026,17435.78,31.02,37.548528,49.63,103.298835,-0.761165,0.982281,0.000003,2.643711
46000,99.93,15633.78,16280.322382,16442.2,37.22,41.735584,55.83,98.570828,-1.359172,0.980601,0.000004,2.981312
47000,97.18,14672.18,15292.900195,15448.62,49.63,47.891095,68.24,94.290391,-2.889609,0.978284,0.000005,3.434603
…,…,…,…,…,…,…,…,…,…,…,…,…
68000,73.24,744.47,773.640335,806.5,6234.9,6393.762909,6607.13,73.173802,-0.066198,0.217343,0.000045,26.1782
70000,74.61,465.29,504.835481,527.33,7940.97,8112.113452,8282.18,75.033817,0.423817,0.151858,0.000035,20.941205
72000,77.68,310.19,332.774787,372.23,9802.13,9927.208156,10143.35,77.197154,-0.482846,0.105443,0.000026,16.253942


## 6. Arbitrage Detection & Visualization

In [ ]:
df_option_with_vola_and_greeks

In [ ]:
# Simplified Arbitrage Detection using pre-calculated call prices
def analyze_arbitrage_conditions(df_with_fitted_prices):
    """Quick arbitrage check using butterfly spreads from fitted call prices"""
    try:
        strikes = df_with_fitted_prices['strike'].to_list()
        call_prices = df_with_fitted_prices['fitted_call_price'].to_list()
        violations = 0
        
        # Check butterfly spreads using adjacent strikes (every 3rd for speed)
        for i in range(3, len(strikes)-3, 3):
            # Get butterfly spread: C(K1) - 2*C(K2) + C(K3)
            butterfly = call_prices[i-3] - 2*call_prices[i] + call_prices[i+3]
            if butterfly < -1e-4:  # Should be >= 0 for no arbitrage
                violations += 1
                
    except Exception:
        violations = 0
    
    # Results
    is_clean = violations == 0
    status = '✅ CLEAN' if is_clean else f'⚠️ {violations} violations'
    return {'violations': violations, 'is_arbitrage_free': is_clean, 'status': status}

print("✅ Arbitrage detection loaded (using pre-calculated prices)")

In [ ]:
# Analyze arbitrage using fitted call prices from DataFrame
print("🔍 ARBITRAGE ANALYSIS USING FITTED PRICES")
print("="*50)

# Use the enhanced DataFrame with fitted call prices
analysis = analyze_arbitrage_conditions(df_option_with_vola_and_greeks)
print(f"📈 FITTED MODEL ARBITRAGE CHECK: {analysis['status']}")
print(f"   - Violations: {analysis['violations']}")
print(f"   - Arbitrage-free: {analysis['is_arbitrage_free']}")

print(f"\n✅ Arbitrage analysis complete using {len(df_option_with_vola_and_greeks)} fitted prices")

In [ ]:
df_option_with_vola_and_greeks.write_csv("option_data.csv")

In [ ]:
df_option_with_vola_and_greeks


In [ ]:
calibration_results.to_pickle("calibration_results.pkl")

In [ ]:
import pickle

with open("calibration_results.pkl", "wb") as f:
    pickle.dump(calibration_results, f)

In [ ]:
calibration_results

In [ ]:
import pickle
with open("calibration_results.pkl", "rb") as f:
        temp = pickle.load(f)